In [2]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime
import math
import calendar
from scipy.optimize import fsolve
from scipy.optimize import minimize
import time

In [3]:
# 确定交易起始日和终止日。

time_period = 1

trade_day = datetime.date(2022, 11, 17) #今天
end_day = trade_day + datetime.timedelta(days=int(time_period*365))

datetime.date(2023, 11, 17)

In [4]:
#用米筐获取银行间市场交易日。账号是老师的

import rqdatac as rq
rq.init()

In [5]:
# 确定计息天数
if calendar.isleap(trade_day.year):
    coupon_days = 366
else:
    coupon_days = 365

In [6]:
# 获取10年期国债利率数据。注意是两份文件拼接，一份数据到2021，另一份到今天
path = 'D:\\2022snowball\\'

bond_rate_old = pd.read_csv(path + 'bond_rate_old.csv', index_col=0, engine='python')
bond_rate_new = pd.read_csv(path + 'bond_rate_new.csv', index_col=0, engine='python')

full_index = list(set(bond_rate_old.index).union(set(bond_rate_new.index)))
full_index.sort()
full_data = pd.DataFrame(index = full_index,columns = bond_rate_new.columns)
full_data.loc[bond_rate_old.index,:] = bond_rate_old
full_data.loc[bond_rate_new.index,:] = bond_rate_new
bond_curves = full_data.copy()#两组数据拼接

# concat也行

#bond_rate_new_cut = bond_rate_new.loc['2021-11-08':, :]
#bond_curves_2 = pd.concat([bond_rate_old, bond_rate_new_cut], axis=0)
#bond_curves_2


In [83]:
# 定价参数：超参数

bond_period = 10

S0 = bond_curves['10'].iloc[0]
K = S0
direction = -1 ##敲出方向
KI_rate = 1.1 ##敲入比例
KO_rate = 1 ##敲出比例
KI = KI_rate * S0 ##KI为敲入价格
KO = KO_rate * S0 ##KO为敲出价格

coupon = 0.13 ##票息率
I = 250000 ##蒙特卡洛模拟路径数，可调整
nominal_principle = 100000000 ##名义本金数
r = 0 ##标的收益率。利率不存在收益率，故设为0
d = 0 ##标的分红率，利率标的为0
rf = 0.018 ##无风险收益率，可调，暂时设为0.023
participation = 1 ##参与率，即内嵌敲入期权的杠杆率,可取200%到300%



In [14]:
# 定价参数2：国债利率波动率。期权的远期波动率由最近一年的历史波动率代替

window = 252
sigma = (bond_curves['10'].diff().iloc[-window-2:-2]/bond_curves['10'].iloc[-window-2:-2]).std() * np.sqrt(252)
sigma

0.10774873509274989

In [7]:
# 历史回测区间，暂且选取为过去一年

#hist_get_trade_days1 = bond_curves.index[-252:]
#hist_get_trade_days = np.array(hist_get_trade_days1)

In [69]:
# 获取历史交易日，提取敲出观察日序列

td_series = rq.get_trading_dates(start_date=trade_day, end_date=end_day)
days_series0 = (pd.Series(td_series) - pd.Series(td_series)[0]).apply(lambda x: x.days) # 所有交易日的天数，为0~365

date_series = pd.Series(td_series).apply(lambda x: x.day)
month_series = pd.Series(td_series).apply(lambda x: x.month)

KO_date_index = pd.Series(0, index=days_series0.index)

for i in range(11):
    current_month = (i+11)%12 + 1  #最后一个月是2023年11月
    select_range = np.where((current_month == month_series) & (date_series >= date_series[0]))[0] #每月17号后所有的交易日期
    if(len(select_range) > 0):
        KO_date_index[select_range[0]] = 1 #若交易日期有，则总是顺延至X月17日的下一个交易日
    else:
        select_range = np.where((current_month == month_series))[0]
        KO_date_index[select_range[-1]] = 1
    
KO_date_index.iloc[-1] = 1 #期末总是敲出

KO_date_series = pd.Series(td_series)[KO_date_index == 1]
KO_day_series = KO_date_series.apply(lambda x: (x - td_series[0]).days) #敲出日序列不用日期，方便比较


In [86]:
## 核心函数：GB10雪球定价（向下敲出、向上敲入，对应direction = -1）

def calc_price_snowball_MC(S_0, K_out, K_in , K, sigma, r, d, rf, coupon_rate, I, days_series0, Knock_out_days_series, state, direction, nominal_principle, participation):
    total_days = days_series0.iloc[-1]
    delta_day = np.array(days_series0 - days_series0.shift(1))  #计算各交易日之间间隔，单位为天
    delta_day[0] = 0

    # 蒙特卡洛模拟
    np.random.seed(1234)
    z = np.random.standard_normal(len(days_series0)*I).reshape(len(days_series0),I)
    S_ratio = (r - d)* delta_day[:, np.newaxis]/365 + K * sigma * z * np.sqrt(delta_day[:, np.newaxis]/365)  # 由假设，利率的随机过程中，漂移率为0，故r和d都为0，
    S_T = S_0 + S_ratio.cumsum(axis = 0) # 累加日间利率变化量
    S_T = S_T.T
    
    S_T = pd.DataFrame(S_T,columns = days_series0,index = range(I))
    

    # 第一类：观察日内敲出
    
    ko = np.sum(np.array(S_T[Knock_out_days_series])*direction > K_out*direction,axis = 1) #计算理论敲出次数
    ko_state = np.sum((1-(np.array(S_T[Knock_out_days_series])*direction > K_out*direction)).cumprod(axis = 1), axis = 1) 
    #累乘，如果先敲出后敲入，这种情况显然不符合实际，故用累乘把1抹平。求和结果为存续的观察日个数
    
    # 提前敲出的期权价值：获得存续日/期限的票息
    value = pd.Series(0, index = range(I))
    value[ko_state < len(Knock_out_days_series)] = nominal_principle * coupon_rate * Knock_out_days_series.iloc[ko_state[ko_state < len(Knock_out_days_series)]]/365 * np.exp(-rf * Knock_out_days_series.iloc[ko_state[ko_state < len(Knock_out_days_series)]]/365)
    

    # 第二类，未敲入也未敲出，期权价值为获得期限内所有票息
    if state == 'NK':
        ko[(np.sum(S_T*direction < K_in*direction,axis = 1) == 0) & (np.sum(np.array(S_T[Knock_out_days_series])*direction > K_out*direction,axis = 1) == 0)] = -1 # 敲入日是每一个交易日，敲出日按预设
        value[ko == -1] = nominal_principle * coupon_rate * total_days/365 * np.exp(-rf * total_days/365)
    
    #第三类，仅敲入未敲出，期权价值为 max(名义本金*(期末国债收益率/期初国债收益率-100%),0)*参与率（杠杆率）
    value[ko == 0] = participation * nominal_principle * np.minimum(0,np.array((S_T[days_series0.iloc[-1]][ko == 0]-K)/S_0)*direction) * np.exp(-rf * total_days/365)
    
    #总价值
    price = np.mean(value)
    return price

In [87]:
state = 'NK'
SnowBall_example = calc_price_snowball_MC(S0, KO, KI, K, sigma, r, d, rf, coupon, I, days_series0, KO_day_series, state, direction, nominal_principle, participation)
print(SnowBall_example)

498361.11084042577


In [53]:
# 读取国债信息数据，为筛选做准备
bond_params = pd.read_csv(path + 'bond_info.csv',index_col = 0)#,encoding = 'gbk',engine='python')
bond_params = bond_params.iloc[:-2,:]
bond_params['起息日期'] = bond_params['起息日期'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
bond_params['计息截止日'] = bond_params['计息截止日'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())


## 对历史交易日的10年期国债计算当时的净价和久期

def calc_hist_bond_and_duration(now_day, last_site, use_old=True): # 交易日，国债代码，是否...
    
    trade_day = now_day
    first_trade_day = datetime.datetime.strptime(bond_curves.index[0], '%Y-%m-%d').date()
    year = now_day.year
    bond_period = 10
    used_bond = []
    
    #计算sigma
    if use_old == True:
        site = last_site
    else:
        bond_params['剩余期限'] = bond_params['计息截止日'].apply(lambda x:(x - now_day).days/365)
        bond_params['差距'] = np.abs(bond_params['剩余期限']- int(bond_period)) # 这句意义不明，3年期还是10年期？
        sorted_bond = (bond_params['差距'][(bond_params['起息日期'] <= trade_day) & (bond_params['起息日期'] > first_trade_day)]).sort_values(ascending = True) #
        minimum_error = np.unique(sorted_bond)[0]
        print(minimum_error)
        count = 0
        selected = pd.DataFrame()
        while len(selected) == 0:
            selected = bond_params[(bond_params['起息日期'] < trade_day) & (bond_params['起息日期'] > first_trade_day) & (bond_params['差距'] == minimum_error)]
            selected['Index'] = selected.index
            count+=1
            minimum_error = np.unique(sorted_bond)[count]
        
        selected_head = selected.apply(lambda x:x.Index[0:2],axis = 1)
        selected_tail = selected.apply(lambda x:x.Index.split('.')[1],axis = 1)
        
        if np.sum((selected_head >= str(year)) & (selected_tail != 'BC')) > 0:
            site = selected.index[np.where((selected_head >= str(year)) & (selected_tail != 'BC'))[0][0]]
        elif np.sum((selected_head == '10') & (selected_tail == 'SZ')) > 0:
            site = selected.index[np.where((selected_head == '10') & (selected_tail == 'SZ'))[0][0]]
        else:
            site = selected.index[0]
        #site = np.argmin(bond_params['差距'][bond_params['起息日期']< trade_day])#得存续
        #bond_params.loc[site,:]
        used_bond.append([site,bond_params.loc[site,'证券简称']])
    

In [54]:
calc_hist_bond_and_duration(trade_day, '220025.IB' ,use_old=False)

0.0
                   证券简称        起息日期  债券期限(年)\r\n[单位] 年       计息截止日  \
证券代码                                                                 
019690.SH        22国债25  2022-11-15               10.0  2032-11-14   
102225.SZ        国债2225  2022-11-15               10.0  2032-11-14   
220025.BC  22附息国债25(柜台)  2022-11-15               10.0  2032-11-14   
220025.IB      22附息国债25  2022-11-15               10.0  2032-11-14   

           票面利率(发行时)\r\n[单位] %     计息基准  每年付息次数  剩余期限   差距      Index  
证券代码                                                                   
019690.SH                  2.8    A/365     2.0  10.0  0.0  019690.SH  
102225.SZ                  2.8    A/365     2.0  10.0  0.0  102225.SZ  
220025.BC                  2.8    A/365     2.0  10.0  0.0  220025.BC  
220025.IB                  2.8  ACT/ACT     2.0  10.0  0.0  220025.IB  


C:\Users\p'c\AppData\Local\Temp\ipykernel_17960\3355186708.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['Index'] = selected.index


In [44]:
bond_params

,证券简称,起息日期,债券期限(年)\r\n[单位] 年,计息截止日,票面利率(发行时)\r\n[单位] %,计息基准,每年付息次数,剩余期限,差距
证券代码,,,,,,,,,
010303.SH,03国债(3),2003-04-17,20.0000,2023-04-16,3.4000,A/365,2.0,0.410959,9.589041
010504.SH,05国债(4),2005-05-15,20.0000,2025-05-14,4.1100,A/365,2.0,2.490411,7.509589
010609.SH,06国债(9),2006-06-26,20.0000,2026-06-25,3.7000,A/365,2.0,3.605479,6.394521
010706.SH,07国债06,2007-05-17,30.0000,2037-05-16,4.2700,A/365,2.0,14.504110,4.504110
010713.SH,07国债13,2007-08-16,20.0000,2027-08-15,4.5200,A/365,2.0,4.745205,5.254795
...,...,...,...,...,...,...,...,...,...
229963.IB,22贴现国债63,2022-11-10,0.0767,2022-12-07,0.8126,ACT/ACT,NaN,0.054795,9.945205
229964.IB,22贴现国债64,2022-11-10,0.1726,2023-01-11,1.1337,ACT/ACT,NaN,0.150685,9.849315
229965.IB,22贴现国债65,2022-11-14,0.4986,2023-05-14,1.6834,ACT/ACT,NaN,0.487671,9.512329
